In [1]:
from langchain.graph_transformers.diffbot import DiffbotGraphTransformer
from langchain.schema import Document

In [2]:
diffbot_nlp = DiffbotGraphTransformer(diffbot_api_key="")

In [3]:
from langchain.graphs import Neo4jGraph

url="bolt://localhost:7687"
username="neo4j"
password="pleaseletmein"

graph = Neo4jGraph(
    url=url,
    username=username, 
    password=password
)

In [4]:
from langchain.document_loaders import WikipediaLoader

# No chunking because
# Maximum of 100,000 characters per document and 1,000,000 total characters per API request.
for query in ["Leonhard Euler", "Barack Obama", "United States", "Neo4j", "Warcraft"]:
    raw_documents = WikipediaLoader(query=query).load()
    graph_documents = diffbot_nlp.transform_documents(raw_documents)
    graph.add_graph_documents(graph_documents, include_source=False)
    print(query)

Leonhard Euler
Barack Obama
United States
Neo4j
Warcraft


In [5]:
graph.refresh_schema()

In [6]:
import os
from langchain.chains import GraphCypherQAChain
from langchain.chat_models import ChatOpenAI

os.environ['OPENAI_API_KEY'] = "sk-"

chain = GraphCypherQAChain.from_llm(
    cypher_llm=ChatOpenAI(temperature=0, model_name="gpt-4"),
    qa_llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo"),
    graph=graph, verbose=True,
    
)
     

In [7]:
chain.run("Where was Leonhard Euler located?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {name: "Leonhard Euler"})-[:ALL_PERSON_LOCATIONS]->(l:Location) RETURN l.name
Full Context:
[{'l.name': 'Basel'}]

> Finished chain.


'Leonhard Euler was located in Basel.'

In [8]:
chain.run("What is Leonhard Euler field of work?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {name: "Leonhard Euler"})-[:FIELD_OF_WORK]->(s:Skill) RETURN s.name
Full Context:
[{'s.name': 'complex analysis'}, {'s.name': 'analytic number theory'}]

> Finished chain.


"Leonhard Euler's field of work includes complex analysis and analytic number theory."

In [9]:
chain.run("Where did Barack Obama attend university?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {name: "Barack Obama"})-[:EDUCATED_AT]->(o:Organization)
RETURN o.name
Full Context:
[{'o.name': 'Maseno National School'}, {'o.name': 'Harvard University'}, {'o.name': 'University of Hawaii'}, {'o.name': 'Punahou School'}, {'o.name': 'Columbia University'}, {'o.name': 'Harvard Law School'}, {'o.name': 'University of Chicago Law School'}]

> Finished chain.


'Barack Obama attended Harvard University.'

In [10]:
chain.run("How many children did the father of Barack Obama have?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {name: "Barack Obama"})-[:PARENT]->(father:Person)-[:CHILD]->(children:Person) RETURN COUNT(children)
Full Context:
[{'COUNT(children)': 2}]

> Finished chain.


'The father of Barack Obama had two children.'

In [11]:
# Close, maybe will work another day
chain.run("What is the evidence for Joe Biden religion?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {name: 'Joe Biden'})-[:RELIGION {evidence: evidence}]->(r:Religion) RETURN evidence


ValueError: Generated Cypher Statement is not valid
{code: Neo.ClientError.Statement.SyntaxError} {message: Variable `evidence` not defined (line 1, column 60 (offset: 59))
"MATCH (p:Person {name: 'Joe Biden'})-[:RELIGION {evidence: evidence}]->(r:Religion) RETURN evidence"
                                                            ^}

In [ ]:
# Close, maybe will work another day
chain.run("What is the Lolo Soetoro field of work and how do we know that?")